# Importing libraries

In [ ]:

# using tensorflow to select models, optimzers their characteristics
import tensorflow as tf
import keras
from sklearn.metrics import cohen_kappa_score, jaccard_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split #to split dataset into 3 parts
import json
import numpy as np   #  to perform mathematic operations
import cv2   # to perform opencv operations
import os   # to navigate through paths of a file
import time
import matplotlib.pyplot as plt   #to display images
from PIL import Image   #to display images
import copy
import h5py
import scipy
from scipy import ndimage
#from lr_utils import load_dataset
#from public_tests import *

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

# Importing dataset

In [ ]:
#!unzip /content/BreaKHis_v2.zip

from sklearn.preprocessing import LabelEncoder

data_dir = 'C:/Users/IIOT CAD/Documents/Meet/BreaKHis_v1'

# list of class labels (categories)
categories = ["adenosis", "fibroadenoma", "phyllodes_tumor", "tubular_adenoma", "ductal_carcinoma", "lobular_carcinoma", "mucinous_carcinoma", "papillary_carcinoma"]

# dictionary to map category names to numeric labels
category_to_label = {category: label for label, category in enumerate(categories)}

# Lists to store images and corresponding labels
dataset = []
labels = []

# the target size for resizing the images
target_size = (224, 224)  

# Looping through each category
for category in categories:
    category_dir = os.path.join(data_dir, category)
    label = category_to_label[category] 

    # Looping through images in the current category
    for dirpath, dirnames, filenames in os.walk(category_dir):
        for filename in filenames:
            if filename.endswith(".png"):
                image_path = os.path.join(dirpath, filename)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Resize the image 
                image = cv2.resize(image, target_size)
                dataset.append(image)
                labels.append(label)  

In [ ]:
# Converting dataset and labels to NumPy arrays
dataset = np.array(dataset)
labels = np.array(labels)
labels = labels.reshape(-1,1)

# Collecting information about dataset loaded

In [ ]:
print(len(dataset))
print(type(dataset))

In [ ]:
type(labels)

In [ ]:
for idx, image_array in enumerate(dataset, start=1):
    height, width, channels = image_array.shape
    #print(f"Image {idx} - Width: {width}, Height: {height}, Channels: {channels}")

In [ ]:
# index for the image
image_index = 1  
normalized_image_array = dataset[image_index]

# Display 
print("Image Shape:", normalized_image_array.shape)
print("Minimum Pixel Value:", np.min(normalized_image_array))
print("Maximum Pixel Value:", np.max(normalized_image_array))

# Preprocessing the image dataset

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_image(image_array, title):
    # Displays the image with a title and hides axes
    plt.figure(figsize=(8, 8))
    plt.imshow(image_array)
    plt.title(title)
    plt.axis('off')
    plt.show()

def preprocess_and_resize(image_array, target_size):
    # Resizes the image to the given size
    resized_image = cv2.resize(image_array, target_size, interpolation=cv2.INTER_NEAREST)

    # Normalizes pixel values
    normalized_image = resized_image / 255.0

    # Sharpens the image using a filter
    sharpening_filter = np.array([[-1, -1, -1],
                                   [-1,  9, -1],
                                   [-1, -1, -1]])
    sharpened_image = cv2.filter2D(normalized_image, -1, sharpening_filter)

    # Blends the sharpened and normalized images
    final_image = 0.8 * normalized_image + 0.2 * sharpened_image

    # Keeps pixel values within range
    final_image = np.clip(final_image, 0, 1)

    return final_image

# Index of the image to display
image_to_display_index = 1
image_to_display = dataset[image_to_display_index]

# Displays the original image
display_image(image_to_display, "Original Image")

# Desired size for all images
target_size = (240, 140)

# Preprocess and resize all images
preprocessed_resized_images = [preprocess_and_resize(image_array, target_size) for image_array in dataset]

# Displays the processed version of the chosen image
display_image(preprocessed_resized_images[image_to_display_index], "Preprocessed, Resized, and Sharpened Image")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_image(image_array, title):
    # Displays the image with a title and hides axes
    plt.figure(figsize=(8, 8))
    plt.imshow(image_array, cmap='gray')  # Using grayscale for segmentation
    plt.title(title)
    plt.axis('off')
    plt.show()

def preprocess_image(image_array, target_size):
    # Resizes the image to the given size
    resized_image = cv2.resize(image_array, target_size)

    # Converts the image to grayscale
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)

    # Segments the image using Otsu's thresholding
    _, segmented_image = cv2.threshold(grayscale_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Normalizes pixel values to the range [0, 1]
    normalized_image = resized_image / 255.0

    # Adjusts segmented image dimensions to match the normalized image
    segmented_image = np.expand_dims(segmented_image, axis=-1)

    # Blends the normalized and segmented images
    final_image = 0.5 * normalized_image + 0.5 * segmented_image / 255.0

    return final_image

# Index of the image to display
image_to_display_index = 10
image_to_display = dataset[image_to_display_index]

# Displays the original image
display_image(image_to_display, "Original Image")

# Sets the target size for all images
target_size = (224, 147)

# Preprocess and segment all images
preprocessed_images = [preprocess_image(image_array, target_size) for image_array in dataset]

# Displays the processed version of the selected image
display_image(preprocessed_images[image_to_display_index], "Preprocessed and Segmented Image")


# Checking if image data is normalized

In [ ]:
# Select the index of the image to examine
image_index = 4  # Change this to the index of the image you want to analyze

# Retrieve the normalized image array for the selected image
normalized_image_array = preprocessed_images[image_index]

# Uncomment this line if using preprocessed and sharpened images
# normalized_image_array = preprocessed_and_sharpened_images[image_index]

# Print details about the normalized image
print("Image Shape:", normalized_image_array.shape)
print("Labels Shape:", labels.shape)
print("Minimum Pixel Value:", np.min(normalized_image_array))
print("Maximum Pixel Value:", np.max(normalized_image_array))


# Splitting the dataset into training, testing, and validation sets

In [ ]:
# Here, we use an 80-10-10 split for train-test-validation respectively
train_images, temp_images, train_labels, temp_labels = train_test_split(preprocessed_images, labels, test_size=0.2, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)
print(len(train_images), len(train_labels), len(val_images), len(val_labels), len(test_images), len(test_labels))

# Assuming train_data and test_data are your training and test datasets

train_images = np.array(train_images)
val_images = np.array(val_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

print("train Shape:", train_images.shape)
print("train labels Shape:", train_labels.shape)
print("val Shape:", val_images.shape)
print("val labels Shape:", val_labels.shape)
print("test Shape:", test_images.shape)
print("test labels Shape:", test_labels.shape)

# Training the model

# DenseNet-201 model with the ADAM

In [ ]:
# Create a Sequential model
model = Sequential()

# Use DenseNet-201 as the base model without its fully connected layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling
model.add(GlobalAveragePooling2D())

# Add custom layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # Adjust the number of classes as needed

# Compile the model with Adam optimizer
adam = Adam(learning_rate=0.01)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model on the test set
test_predictions = np.argmax(model.predict(test_images), axis=-1)

# Calculate evaluation metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')

# Store evaluation results
test_accuracy_DenseNet201_ADAM_01 = test_accuracy
test_precision_DenseNet201_ADAM_01 = test_precision
test_recall_DenseNet201_ADAM_01 = test_recall
test_cohen_kappa_DenseNet201_ADAM_01 = test_cohen_kappa
test_jaccard_index_DenseNet201_ADAM_01 = test_jaccard_index

# Print evaluation results
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Save the accuracy plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_ADAM_01.png')
plt.savefig(plot_filename)

# Compute and display the confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_ADAM_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()


In [ ]:
# Create a Sequential model
model = Sequential()

# Load DenseNet-201 as the base model without its top layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling for feature extraction
model.add(GlobalAveragePooling2D())

# Add custom layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # Number of classes in your dataset

# Compile the model with the Adam optimizer
adam = Adam(learning_rate=0.001)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model structure
model.summary()

# Train the model with training and validation data
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model on the test dataset
test_predictions = np.argmax(model.predict(test_images), axis=-1)

# Calculate evaluation metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')

# Save the metrics for this configuration
test_accuracy_DenseNet201_ADAM_001 = test_accuracy
test_precision_DenseNet201_ADAM_001 = test_precision
test_recall_DenseNet201_ADAM_001 = test_recall
test_cohen_kappa_DenseNet201_ADAM_001 = test_cohen_kappa
test_jaccard_index_DenseNet201_ADAM_001 = test_jaccard_index

# Print the evaluation results
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Save the accuracy plot to a directory
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_ADAM_001.png')
plt.savefig(plot_filename)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_ADAM_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()


In [ ]:
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
adam = Adam(learning_rate=0.0001)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_accuracy_DenseNet201_ADAM_0001 = test_accuracy
test_precision_DenseNet201_ADAM_0001 = test_precision
test_recall_DenseNet201_ADAM_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_ADAM_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_ADAM_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_ADAM_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_ADAM_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# DenseNet-201 model with the SGD

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.01)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_SGD = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_accuracy_DenseNet201_SGD_01 = test_accuracy
test_precision_DenseNet201_SGD_01 = test_precision
test_recall_DenseNet201_SGD_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_SGD_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_SGD_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_SGD_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_SGD_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_SGD = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_accuracy_DenseNet201_SGD_001 = test_accuracy
test_precision_DenseNet201_SGD_001 = test_precision
test_recall_DenseNet201_SGD_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_SGD_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_SGD_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_SGD_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_SGD_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.0001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_SGD = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_accuracy_DenseNet201_SGD_0001 = test_accuracy
test_precision_DenseNet201_SGD_0001 = test_precision
test_recall_DenseNet201_SGD_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_SGD_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_SGD_0001 = test_jaccard_index

print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_SGD_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_SGD_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# DenseNet-201 model with the RMSprop

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.01)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_RMSprop_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_precision_DenseNet201_RMSprop_01 = test_precision
test_recall_DenseNet201_RMSprop_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_RMSprop_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_RMSprop_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_RMSprop_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_RMSprop_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_RMSprop_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_precision_DenseNet201_RMSprop_001 = test_precision
test_recall_DenseNet201_RMSprop_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_RMSprop_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_RMSprop_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_RMSprop.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_RMSprop.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.0001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_DenseNet201_RMSprop_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

test_precision_DenseNet201_RMSprop_0001 = test_precision
test_recall_DenseNet201_RMSprop_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_DenseNet201_RMSprop_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_DenseNet201_RMSprop_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_DenseNet201_RMSprop_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_DenseNet201_RMSprop_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# COMPARATIVE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# List of optimizers
optimizers = ['Adam', 'SGD', 'RMSprop']

# List of learning rates
learning_rates = ['LR: 0.01', 'LR: 0.001', 'LR: 0.0001']

# List of corresponding accuracy values
accuracies = [
    [test_accuracy_DenseNet201_ADAM_01, test_accuracy_DenseNet201_ADAM_001, test_accuracy_DenseNet201_ADAM_0001],
    [test_accuracy_DenseNet201_SGD_01, test_accuracy_DenseNet201_SGD_001, test_accuracy_DenseNet201_SGD_0001],
    [test_accuracy_DenseNet201_RMSprop_01, test_accuracy_DenseNet201_RMSprop_001, test_accuracy_DenseNet201_RMSprop_0001]
]

# Set the width of each bar
bar_width = 0.9

# Set the distance between bars for different optimizers
distance_between_optimizers = 0.6

# Set the distance between optimizer names on the x-axis
distance_between_names = 4

# Define unique colors for each learning rate
lr_colors = {'LR: 0.01': 'skyblue', 'LR: 0.001': 'brown', 'LR: 0.0001': 'lightgreen'}

# Plot grouped bar graph
fig, ax = plt.subplots()

# Plot bars for each optimizer
for i, optimizer in enumerate(optimizers):
    positions = np.arange(len(learning_rates)) + i * (len(learning_rates) * (bar_width + distance_between_optimizers))
    for j, lr in enumerate(learning_rates):
        color = lr_colors[lr]
        ax.bar(positions[j], accuracies[i][j], bar_width, label=f'{lr}', color=color)

# Add labels and title
ax.set_xlabel('Optimizers')
ax.set_ylabel('Test Accuracy')
ax.set_title('Test Accuracy vs Optimizers')
ax.set_xticks(np.arange(len(optimizers)) * distance_between_names + (len(learning_rates) - 1) * (bar_width + distance_between_optimizers) / 2)
ax.set_xticklabels(optimizers)

# Add legend with adjusted position and ordered values
handles, labels = ax.get_legend_handles_labels()
unique_labels = list(set(labels))
# Order values in the legend
ordered_values = ['0.01', '0.001', '0.0001']
# Create a dictionary to map old labels to new labels
label_mapping = {f'LR: {value}': f'LR: {value}' for value in ordered_values}
new_labels = [label_mapping[label] for label in labels]
# Plot the legend
ax.legend(handles[:len(learning_rates)], new_labels, title='Learning Rate', bbox_to_anchor=(1, 1), loc='upper left')

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'comparative_analysis_DenseNet201.png')
plt.savefig(plot_filename, bbox_inches='tight')

# Show the plot
plt.show()


# InceptionV3 with Adam optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.01)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_ADAM_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_ADAM_01 = test_precision
test_recall_InceptionV3_ADAM_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_ADAM_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_ADAM_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_ADAM_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_ADAM_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_ADAM_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_ADAM_001 = test_precision
test_recall_InceptionV3_ADAM_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_ADAM_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_ADAM_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_ADAM_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_ADAM_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.0001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_ADAM_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_ADAM_0001 = test_precision
test_recall_InceptionV3_ADAM_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_ADAM_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_ADAM_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_ADAM_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_ADAM_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# InceptionV3 with SGD optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop 
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.01)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_SGD_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_SGD_01 = test_precision
test_recall_InceptionV3_SGD_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_SGD_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_SGD_01 = test_jaccard_index

print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_SGD_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_SGD_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop 
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_SGD_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_SGD_001 = test_precision
test_recall_InceptionV3_SGD_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_SGD_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_SGD_001 = test_jaccard_index

print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_SGD_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_SGD_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop 
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.0001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_SGD_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_SGD_0001 = test_precision
test_recall_InceptionV3_SGD_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_SGD_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_SGD_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_SGD_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_SGD_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# InceptionV3 with RMSprop optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.01)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_RMSprop_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_RMSprop_01 = test_precision
test_recall_InceptionV3_RMSprop_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_RMSprop_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_RMSprop_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_RMSprop_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_RMSprop_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_RMSprop_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_RMSprop_001 = test_precision
test_recall_InceptionV3_RMSprop_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_RMSprop_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_RMSprop_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_RMSprop_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_RMSprop_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.0001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_InceptionV3_RMSprop_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_InceptionV3_RMSprop_0001 = test_precision
test_recall_InceptionV3_RMSprop_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_InceptionV3_RMSprop_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_InceptionV3_RMSprop_0001 = test_jaccard_index

print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_InceptionV3_RMSprop_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_InceptionV3_RMSprop_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# COMPARATIVE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# List of optimizers
optimizers = ['Adam', 'SGD', 'RMSprop']

# List of learning rates
learning_rates = ['LR: 0.01', 'LR: 0.001', 'LR: 0.0001']

# List of corresponding accuracy values
accuracies = [
    [test_accuracy_InceptionV3_ADAM_01, test_accuracy_InceptionV3_ADAM_001, test_accuracy_InceptionV3_ADAM_0001],
    [test_accuracy_InceptionV3_SGD_01, test_accuracy_InceptionV3_SGD_001, test_accuracy_InceptionV3_SGD_0001],
    [test_accuracy_InceptionV3_RMSprop_01, test_accuracy_InceptionV3_RMSprop_001, test_accuracy_InceptionV3_RMSprop_0001]
]

# Set the width of each bar
bar_width = 0.9

# Set the distance between bars for different optimizers
distance_between_optimizers = 0.6

# Set the distance between optimizer names on the x-axis
distance_between_names = 4

# Define unique colors for each learning rate
lr_colors = {'LR: 0.01': 'skyblue', 'LR: 0.001': 'brown', 'LR: 0.0001': 'lightgreen'}

# Plot grouped bar graph
fig, ax = plt.subplots()

# Plot bars for each optimizer
for i, optimizer in enumerate(optimizers):
    positions = np.arange(len(learning_rates)) + i * (len(learning_rates) * (bar_width + distance_between_optimizers))
    for j, lr in enumerate(learning_rates):
        color = lr_colors[lr]
        ax.bar(positions[j], accuracies[i][j], bar_width, label=f'{lr}', color=color)

# Add labels and title
ax.set_xlabel('Optimizers')
ax.set_ylabel('Test Accuracy')
ax.set_title('Test Accuracy vs Optimizers')
ax.set_xticks(np.arange(len(optimizers)) * distance_between_names + (len(learning_rates) - 1) * (bar_width + distance_between_optimizers) / 2)
ax.set_xticklabels(optimizers)

# Add legend with adjusted position and ordered values
handles, labels = ax.get_legend_handles_labels()
unique_labels = list(set(labels))
# Order values in the legend
ordered_values = ['0.01', '0.001', '0.0001']
# Create a dictionary to map old labels to new labels
label_mapping = {f'LR: {value}': f'LR: {value}' for value in ordered_values}
new_labels = [label_mapping[label] for label in labels]
# Plot the legend
ax.legend(handles[:len(learning_rates)], new_labels, title='Learning Rate', bbox_to_anchor=(1, 1), loc='upper left')

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'comparative_analysis_InceptionV3.png')
plt.savefig(plot_filename, bbox_inches='tight')

# Show the plot
plt.show()


# MobileNetV2 with ADAM optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.01)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_ADAM_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_ADAM_01 = test_precision
test_recall_MobileNetV2_ADAM_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_ADAM_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_ADAM_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_ADAM_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_ADAM_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_ADAM_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_ADAM_001 = test_precision
test_recall_MobileNetV2_ADAM_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_ADAM_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_ADAM_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_ADAM_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_ADAM_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.0001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_ADAM_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_ADAM_0001 = test_precision
test_recall_MobileNetV2_ADAM_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_ADAM_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_ADAM_0001 = test_jaccard_index

print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_ADAM_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_ADAM_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# MobileNetV2 with SGD optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.01)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_SGD_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_SGD_01 = test_precision
test_recall_MobileNetV2_SGD_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_SGD_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_SGD_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_SGD_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_SGD_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_SGD_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_SGD_001 = test_precision
test_recall_MobileNetV2_SGD_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_SGD_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_SGD_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_SGD_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_SGD_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.0001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_SGD_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_SGD_0001 = test_precision
test_recall_MobileNetV2_SGD_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_SGD_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_SGD_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_SGD_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_SGD_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# MobileNetV2 with RMSprop optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.01)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_RMSprop_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_RMSprop_01 = test_precision
test_recall_MobileNetV2_RMSprop_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_RMSprop_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_RMSprop_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_RMSprop_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_RMSprop_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_RMSprop_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_RMSprop_001 = test_precision
test_recall_MobileNetV2_RMSprop_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_RMSprop_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_RMSprop_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_RMSprop_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_RMSprop_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.0001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_MobileNetV2_RMSprop_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_MobileNetV2_RMSprop_0001 = test_precision
test_recall_MobileNetV2_RMSprop_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_MobileNetV2_RMSprop_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_MobileNetV2_RMSprop_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_MobileNetV2_RMSprop_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_MobileNetV2_RMSprop_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# COMPARATIVE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# List of optimizers
optimizers = ['Adam', 'SGD', 'RMSprop']

# List of learning rates
learning_rates = ['LR: 0.01', 'LR: 0.001', 'LR: 0.0001']

# List of corresponding accuracy values
accuracies = [
    [test_accuracy_MobileNetV2_ADAM_01, test_accuracy_MobileNetV2_ADAM_001, test_accuracy_MobileNetV2_ADAM_0001],
    [test_accuracy_MobileNetV2_SGD_01, test_accuracy_MobileNetV2_SGD_001, test_accuracy_MobileNetV2_SGD_0001],
    [test_accuracy_MobileNetV2_RMSprop_01, test_accuracy_MobileNetV2_RMSprop_001, test_accuracy_MobileNetV2_RMSprop_0001]
]

# Set the width of each bar
bar_width = 0.9

# Set the distance between bars for different optimizers
distance_between_optimizers = 0.6

# Set the distance between optimizer names on the x-axis
distance_between_names = 4

# Define unique colors for each learning rate
lr_colors = {'LR: 0.01': 'skyblue', 'LR: 0.001': 'brown', 'LR: 0.0001': 'lightgreen'}

# Plot grouped bar graph
fig, ax = plt.subplots()

# Plot bars for each optimizer
for i, optimizer in enumerate(optimizers):
    positions = np.arange(len(learning_rates)) + i * (len(learning_rates) * (bar_width + distance_between_optimizers))
    for j, lr in enumerate(learning_rates):
        color = lr_colors[lr]
        ax.bar(positions[j], accuracies[i][j], bar_width, label=f'{lr}', color=color)

# Add labels and title
ax.set_xlabel('Optimizers')
ax.set_ylabel('Test Accuracy')
ax.set_title('Test Accuracy vs Optimizers')
ax.set_xticks(np.arange(len(optimizers)) * distance_between_names + (len(learning_rates) - 1) * (bar_width + distance_between_optimizers) / 2)
ax.set_xticklabels(optimizers)

# Add legend with adjusted position and ordered values
handles, labels = ax.get_legend_handles_labels()
unique_labels = list(set(labels))
# Order values in the legend
ordered_values = ['0.01', '0.001', '0.0001']
# Create a dictionary to map old labels to new labels
label_mapping = {f'LR: {value}': f'LR: {value}' for value in ordered_values}
new_labels = [label_mapping[label] for label in labels]
# Plot the legend
ax.legend(handles[:len(learning_rates)], new_labels, title='Learning Rate', bbox_to_anchor=(1, 1), loc='upper left')

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'comparative_analysis_MobileNetV2.png')
plt.savefig(plot_filename, bbox_inches='tight')

# Show the plot
plt.show()


# ResNet50 with Adam optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.01)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_ADAM_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_ADAM_01 = test_precision
test_recall_ResNet50_ADAM_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_ADAM_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_ADAM_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_ADAM_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_ADAM_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_ADAM_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_ADAM_001 = test_precision
test_recall_ResNet50_ADAM_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_ADAM_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_ADAM_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_ADAM_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_ADAM_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.0001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_ADAM_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_ADAM_0001 = test_precision
test_recall_ResNet50_ADAM_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_ADAM_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_ADAM_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_ADAM_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_ADAM_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# ResNet50 with SGD optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.01)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_SGD_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_SGD_01 = test_precision
test_recall_ResNet50_SGD_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_SGD_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_SGD_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_SGD_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_SGD_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_SGD_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_SGD_001 = test_precision
test_recall_ResNet50_SGD_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_SGD_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_SGD_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_SGD_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_SGD_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
SGD = SGD(learning_rate=0.0001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_SGD_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_SGD_0001 = test_precision
test_recall_ResNet50_SGD_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_SGD_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_SGD_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_SGD_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_SGD_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# ResNet50 with RMSprop optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.01)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_RMSprop_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_RMSprop_01 = test_precision
test_recall_ResNet50_RMSprop_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_RMSprop_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_RMSprop_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_RMSprop_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_RMSprop_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_RMSprop_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_RMSprop_001 = test_precision
test_recall_ResNet50_RMSprop_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_RMSprop_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_RMSprop_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_RMSprop_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_RMSprop_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.0001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_ResNet50_RMSprop_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_ResNet50_RMSprop_0001 = test_precision
test_recall_ResNet50_RMSprop_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_ResNet50_RMSprop_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_ResNet50_RMSprop_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_ResNet50_RMSprop_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_ResNet50_RMSprop_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# COMPARATIVE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# List of optimizers
optimizers = ['Adam', 'SGD', 'RMSprop']

# List of learning rates
learning_rates = ['LR: 0.01', 'LR: 0.001', 'LR: 0.0001']

# List of corresponding accuracy values
accuracies = [
    [test_accuracy_ResNet50_ADAM_01, test_accuracy_ResNet50_ADAM_001, test_accuracy_ResNet50_ADAM_0001],
    [test_accuracy_ResNet50_SGD_01, test_accuracy_ResNet50_SGD_001, test_accuracy_ResNet50_SGD_0001],
    [test_accuracy_ResNet50_RMSprop_01, test_accuracy_ResNet50_RMSprop_001, test_accuracy_ResNet50_RMSprop_0001]
]

# Set the width of each bar
bar_width = 0.9

# Set the distance between bars for different optimizers
distance_between_optimizers = 0.6

# Set the distance between optimizer names on the x-axis
distance_between_names = 4

# Define unique colors for each learning rate
lr_colors = {'LR: 0.01': 'skyblue', 'LR: 0.001': 'brown', 'LR: 0.0001': 'lightgreen'}

# Plot grouped bar graph
fig, ax = plt.subplots()

# Plot bars for each optimizer
for i, optimizer in enumerate(optimizers):
    positions = np.arange(len(learning_rates)) + i * (len(learning_rates) * (bar_width + distance_between_optimizers))
    for j, lr in enumerate(learning_rates):
        color = lr_colors[lr]
        ax.bar(positions[j], accuracies[i][j], bar_width, label=f'{lr}', color=color)

# Add labels and title
ax.set_xlabel('Optimizers')
ax.set_ylabel('Test Accuracy')
ax.set_title('Test Accuracy vs Optimizers')
ax.set_xticks(np.arange(len(optimizers)) * distance_between_names + (len(learning_rates) - 1) * (bar_width + distance_between_optimizers) / 2)
ax.set_xticklabels(optimizers)

# Add legend with adjusted position and ordered values
handles, labels = ax.get_legend_handles_labels()
unique_labels = list(set(labels))
# Order values in the legend
ordered_values = ['0.01', '0.001', '0.0001']
# Create a dictionary to map old labels to new labels
label_mapping = {f'LR: {value}': f'LR: {value}' for value in ordered_values}
new_labels = [label_mapping[label] for label in labels]
# Plot the legend
ax.legend(handles[:len(learning_rates)], new_labels, title='Learning Rate', bbox_to_anchor=(1, 1), loc='upper left')

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'comparative_analysis_ResNet50.png')
plt.savefig(plot_filename, bbox_inches='tight')

# Show the plot
plt.show()


# VGG19 with Adam optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
Adam = Adam(learning_rate=0.01)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_ADAM_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_ADAM_01 = test_precision
test_recall_VGG19_ADAM_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_ADAM_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_ADAM_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_ADAM_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_ADAM_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_ADAM_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_ADAM_001 = test_precision
test_recall_VGG19_ADAM_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_ADAM_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_ADAM_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_ADAM_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_ADAM_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
Adam = Adam(learning_rate=0.0001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_ADAM_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_ADAM_0001 = test_precision
test_recall_VGG19_ADAM_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_ADAM_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_ADAM_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_ADAM_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_ADAM_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# VGG19 with SGD optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
SGD = SGD(learning_rate=0.01)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_SGD_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_SGD_01 = test_precision
test_recall_VGG19_SGD_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_SGD_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_SGD_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_SGD_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_SGD_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
SGD = SGD(learning_rate=0.001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_SGD_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_SGD_001 = test_precision
test_recall_VGG19_SGD_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_SGD_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_SGD_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_SGD_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_SGD_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
SGD = SGD(learning_rate=0.0001)
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_SGD_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_SGD_0001 = test_precision
test_recall_VGG19_SGD_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_SGD_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_SGD_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_SGD_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_SGD_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# VGG19 with RMSprop optimizer

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with Adam optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.01)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_RMSprop_01 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_RMSprop_01 = test_precision
test_recall_VGG19_RMSprop_01 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_RMSprop_01 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_RMSprop_01 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_RMSprop_01.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_RMSprop_01.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_RMSprop_001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_RMSprop_001 = test_precision
test_recall_VGG19_RMSprop_001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_RMSprop_001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_RMSprop_001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_RMSprop_001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_RMSprop_001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet201, InceptionV3, MobileNetV2, ResNet50, VGG19
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# Create a Sequential model
model = Sequential()

# Load the pre-trained DenseNet-201 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(147, 224, 3))
model.add(base_model)

# Add global average pooling layer
model.add(GlobalAveragePooling2D())

# Add custom top layers for classification
model.add(Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(8, activation='softmax'))  # 'num_classes' is the number of classes in your dataset

# Compile the model with optimizer and appropriate loss function
RMSprop = RMSprop(learning_rate=0.0001)
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=30,
                    validation_data=(val_images, val_labels))

# Evaluate the model
test_predictions = np.argmax(model.predict(test_images), axis=-1)
#test_labels = np.argmax(test_labels, axis=-1)  # Only needed if one-hot encoded labels are used

test_accuracy = accuracy_score(test_labels, test_predictions)
test_accuracy_VGG19_RMSprop_0001 = test_accuracy
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_precision_VGG19_RMSprop_0001 = test_precision
test_recall_VGG19_RMSprop_0001 = test_recall
test_cohen_kappa = cohen_kappa_score(test_labels, test_predictions)
test_cohen_kappa_VGG19_RMSprop_0001 = test_cohen_kappa
test_jaccard_index = jaccard_score(test_labels, test_predictions, average='weighted')
test_jaccard_index_VGG19_RMSprop_0001 = test_jaccard_index
print("Test accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)
print("Test F1 score:", test_f1)
print("Cohen's Kappa:", test_cohen_kappa)
print("Jaccard Index:", test_jaccard_index)

# Plot the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

# Create a directory named 'results'
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)

# Save the plot inside the 'results' directory
plot_filename = os.path.join(output_directory, 'accuracy_plot_VGG19_RMSprop_0001.png')
plt.savefig(plot_filename)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

# Display the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save the confusion matrix plot inside the 'results' directory
confusion_matrix_filename = os.path.join(output_directory, 'confusion_matrix_VGG19_RMSprop_0001.png')
plt.savefig(confusion_matrix_filename, bbox_inches='tight')
plt.show()

# COMPARATIVE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# List of optimizers
optimizers = ['Adam', 'SGD', 'RMSprop']

# List of learning rates
learning_rates = ['LR: 0.01', 'LR: 0.001', 'LR: 0.0001']

# List of corresponding accuracy values
accuracies = [
    [test_accuracy_VGG19_ADAM_01, test_accuracy_VGG19_ADAM_001, test_accuracy_VGG19_ADAM_0001],
    [test_accuracy_VGG19_SGD_01, test_accuracy_VGG19_SGD_001, test_accuracy_VGG19_SGD_0001],
    [test_accuracy_VGG19_RMSprop_01, test_accuracy_VGG19_RMSprop_001, test_accuracy_VGG19_RMSprop_0001]
]

# Set the width of each bar
bar_width = 0.9

# Set the distance between bars for different optimizers
distance_between_optimizers = 0.6

# Set the distance between optimizer names on the x-axis
distance_between_names = 4

# Define unique colors for each learning rate
lr_colors = {'LR: 0.01': 'skyblue', 'LR: 0.001': 'brown', 'LR: 0.0001': 'lightgreen'}

# Plot grouped bar graph
fig, ax = plt.subplots()

# Plot bars for each optimizer
for i, optimizer in enumerate(optimizers):
    positions = np.arange(len(learning_rates)) + i * (len(learning_rates) * (bar_width + distance_between_optimizers))
    for j, lr in enumerate(learning_rates):
        color = lr_colors[lr]
        ax.bar(positions[j], accuracies[i][j], bar_width, label=f'{lr}', color=color)

# Add labels and title
ax.set_xlabel('Optimizers')
ax.set_ylabel('Test Accuracy')
ax.set_title('Test Accuracy vs Optimizers')
ax.set_xticks(np.arange(len(optimizers)) * distance_between_names + (len(learning_rates) - 1) * (bar_width + distance_between_optimizers) / 2)
ax.set_xticklabels(optimizers)

# Add legend with adjusted position and ordered values
handles, labels = ax.get_legend_handles_labels()
unique_labels = list(set(labels))
# Order values in the legend
ordered_values = ['0.01', '0.001', '0.0001']
# Create a dictionary to map old labels to new labels
label_mapping = {f'LR: {value}': f'LR: {value}' for value in ordered_values}
new_labels = [label_mapping[label] for label in labels]
# Plot the legend
ax.legend(handles[:len(learning_rates)], new_labels, title='Learning Rate', bbox_to_anchor=(1, 1), loc='upper left')

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'comparative_analysis_VGG19.png')
plt.savefig(plot_filename, bbox_inches='tight')

# Show the plot
plt.show()


# COMPARATIVE ANALYSIS OF ALL 45 MODELS

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# Average metric formulas for DenseNet201
average_test_accuracy_DenseNet201 = (test_accuracy_DenseNet201_ADAM_01 + test_accuracy_DenseNet201_ADAM_001 + test_accuracy_DenseNet201_ADAM_0001 +
                                     test_accuracy_DenseNet201_SGD_01 + test_accuracy_DenseNet201_SGD_001 + test_accuracy_DenseNet201_SGD_0001 +
                                     test_accuracy_DenseNet201_RMSprop_01 + test_accuracy_DenseNet201_RMSprop_001 + test_accuracy_DenseNet201_RMSprop_0001) / 9

average_test_precision_DenseNet201 = (test_precision_DenseNet201_ADAM_01 + test_precision_DenseNet201_ADAM_001 + test_precision_DenseNet201_ADAM_0001 +
                                      test_precision_DenseNet201_SGD_01 + test_precision_DenseNet201_SGD_001 + test_precision_DenseNet201_SGD_0001 +
                                      test_precision_DenseNet201_RMSprop_01 + test_precision_DenseNet201_RMSprop_001 + test_precision_DenseNet201_RMSprop_0001) / 9

average_test_recall_DenseNet201 = (test_recall_DenseNet201_ADAM_01 + test_recall_DenseNet201_ADAM_001 + test_recall_DenseNet201_ADAM_0001 +
                                   test_recall_DenseNet201_SGD_01 + test_recall_DenseNet201_SGD_001 + test_recall_DenseNet201_SGD_0001 +
                                   test_recall_DenseNet201_RMSprop_01 + test_recall_DenseNet201_RMSprop_001 + test_recall_DenseNet201_RMSprop_0001) / 9

average_test_cohen_kappa_DenseNet201 = (test_cohen_kappa_DenseNet201_ADAM_01 + test_cohen_kappa_DenseNet201_ADAM_001 + test_cohen_kappa_DenseNet201_ADAM_0001 +
                                   test_cohen_kappa_DenseNet201_SGD_01 + test_cohen_kappa_DenseNet201_SGD_001 + test_cohen_kappa_DenseNet201_SGD_0001 +
                                   test_cohen_kappa_DenseNet201_RMSprop_01 + test_cohen_kappa_DenseNet201_RMSprop_001 + test_cohen_kappa_DenseNet201_RMSprop_0001) / 9

average_test_jaccard_index_DenseNet201 = (test_jaccard_index_DenseNet201_ADAM_01 + test_jaccard_index_DenseNet201_ADAM_001 + test_jaccard_index_DenseNet201_ADAM_0001 +
                                   test_jaccard_index_DenseNet201_SGD_01 + test_jaccard_index_DenseNet201_SGD_001 + test_jaccard_index_DenseNet201_SGD_0001 +
                                   test_jaccard_index_DenseNet201_RMSprop_01 + test_jaccard_index_DenseNet201_RMSprop_001 + test_jaccard_index_DenseNet201_RMSprop_0001) / 9

# Average metric formulas for InceptionV3
average_test_accuracy_InceptionV3 = (test_accuracy_InceptionV3_ADAM_01 + test_accuracy_InceptionV3_ADAM_001 + test_accuracy_InceptionV3_ADAM_0001 +
                                      test_accuracy_InceptionV3_SGD_01 + test_accuracy_InceptionV3_SGD_001 + test_accuracy_InceptionV3_SGD_0001 +
                                      test_accuracy_InceptionV3_RMSprop_01 + test_accuracy_InceptionV3_RMSprop_001 + test_accuracy_InceptionV3_RMSprop_0001) / 9

average_test_precision_InceptionV3 = (test_precision_InceptionV3_ADAM_01 + test_precision_InceptionV3_ADAM_001 + test_precision_InceptionV3_ADAM_0001 +
                                       test_precision_InceptionV3_SGD_01 + test_precision_InceptionV3_SGD_001 + test_precision_InceptionV3_SGD_0001 +
                                       test_precision_InceptionV3_RMSprop_01 + test_precision_InceptionV3_RMSprop_001 + test_precision_InceptionV3_RMSprop_0001) / 9

average_test_recall_InceptionV3 = (test_recall_InceptionV3_ADAM_01 + test_recall_InceptionV3_ADAM_001 + test_recall_InceptionV3_ADAM_0001 +
                                    test_recall_InceptionV3_SGD_01 + test_recall_InceptionV3_SGD_001 + test_recall_InceptionV3_SGD_0001 +
                                    test_recall_InceptionV3_RMSprop_01 + test_recall_InceptionV3_RMSprop_001 + test_recall_InceptionV3_RMSprop_0001) / 9

average_test_cohen_kappa_InceptionV3 = (test_cohen_kappa_InceptionV3_ADAM_01 + test_cohen_kappa_InceptionV3_ADAM_001 + test_cohen_kappa_InceptionV3_ADAM_0001 +
                                   test_cohen_kappa_InceptionV3_SGD_01 + test_cohen_kappa_InceptionV3_SGD_001 + test_cohen_kappa_InceptionV3_SGD_0001 +
                                   test_cohen_kappa_InceptionV3_RMSprop_01 + test_cohen_kappa_InceptionV3_RMSprop_001 + test_cohen_kappa_InceptionV3_RMSprop_0001) / 9

average_test_jaccard_index_InceptionV3 = (test_jaccard_index_InceptionV3_ADAM_01 + test_jaccard_index_InceptionV3_ADAM_001 + test_jaccard_index_InceptionV3_ADAM_0001 +
                                   test_jaccard_index_InceptionV3_SGD_01 + test_jaccard_index_InceptionV3_SGD_001 + test_jaccard_index_InceptionV3_SGD_0001 +
                                   test_jaccard_index_InceptionV3_RMSprop_01 + test_jaccard_index_InceptionV3_RMSprop_001 + test_jaccard_index_InceptionV3_RMSprop_0001) / 9

# Average metric formulas for MobileNetV2
average_test_accuracy_MobileNetV2 = (test_accuracy_MobileNetV2_ADAM_01 + test_accuracy_MobileNetV2_ADAM_001 + test_accuracy_MobileNetV2_ADAM_0001 +
                                      test_accuracy_MobileNetV2_SGD_01 + test_accuracy_MobileNetV2_SGD_001 + test_accuracy_MobileNetV2_SGD_0001 +
                                      test_accuracy_MobileNetV2_RMSprop_01 + test_accuracy_MobileNetV2_RMSprop_001 + test_accuracy_MobileNetV2_RMSprop_0001) / 9

average_test_precision_MobileNetV2 = (test_precision_MobileNetV2_ADAM_01 + test_precision_MobileNetV2_ADAM_001 + test_precision_MobileNetV2_ADAM_0001 +
                                       test_precision_MobileNetV2_SGD_01 + test_precision_MobileNetV2_SGD_001 + test_precision_MobileNetV2_SGD_0001 +
                                       test_precision_MobileNetV2_RMSprop_01 + test_precision_MobileNetV2_RMSprop_001 + test_precision_MobileNetV2_RMSprop_0001) / 9

average_test_recall_MobileNetV2 = (test_recall_MobileNetV2_ADAM_01 + test_recall_MobileNetV2_ADAM_001 + test_recall_MobileNetV2_ADAM_0001 +
                                    test_recall_MobileNetV2_SGD_01 + test_recall_MobileNetV2_SGD_001 + test_recall_MobileNetV2_SGD_0001 +
                                    test_recall_MobileNetV2_RMSprop_01 + test_recall_MobileNetV2_RMSprop_001 + test_recall_MobileNetV2_RMSprop_0001) / 9

average_test_cohen_kappa_MobileNetV2 = (test_cohen_kappa_MobileNetV2_ADAM_01 + test_cohen_kappa_MobileNetV2_ADAM_001 + test_cohen_kappa_MobileNetV2_ADAM_0001 +
                                   test_cohen_kappa_MobileNetV2_SGD_01 + test_cohen_kappa_MobileNetV2_SGD_001 + test_cohen_kappa_MobileNetV2_SGD_0001 +
                                   test_cohen_kappa_MobileNetV2_RMSprop_01 + test_cohen_kappa_MobileNetV2_RMSprop_001 + test_cohen_kappa_MobileNetV2_RMSprop_0001) / 9

average_test_jaccard_index_MobileNetV2 = (test_jaccard_index_MobileNetV2_ADAM_01 + test_jaccard_index_MobileNetV2_ADAM_001 + test_jaccard_index_MobileNetV2_ADAM_0001 +
                                   test_jaccard_index_MobileNetV2_SGD_01 + test_jaccard_index_MobileNetV2_SGD_001 + test_jaccard_index_MobileNetV2_SGD_0001 +
                                   test_jaccard_index_MobileNetV2_RMSprop_01 + test_jaccard_index_MobileNetV2_RMSprop_001 + test_jaccard_index_MobileNetV2_RMSprop_0001) / 9

# Average metric formulas for ResNet50
average_test_accuracy_ResNet50 = (test_accuracy_ResNet50_ADAM_01 + test_accuracy_ResNet50_ADAM_001 + test_accuracy_ResNet50_ADAM_0001 +
                                   test_accuracy_ResNet50_SGD_01 + test_accuracy_ResNet50_SGD_001 + test_accuracy_ResNet50_SGD_0001 +
                                   test_accuracy_ResNet50_RMSprop_01 + test_accuracy_ResNet50_RMSprop_001 + test_accuracy_ResNet50_RMSprop_0001) / 9

average_test_precision_ResNet50 = (test_precision_ResNet50_ADAM_01 + test_precision_ResNet50_ADAM_001 + test_precision_ResNet50_ADAM_0001 +
                                    test_precision_ResNet50_SGD_01 + test_precision_ResNet50_SGD_001 + test_precision_ResNet50_SGD_0001 +
                                    test_precision_ResNet50_RMSprop_01 + test_precision_ResNet50_RMSprop_001 + test_precision_ResNet50_RMSprop_0001) / 9

average_test_recall_ResNet50 = (test_recall_ResNet50_ADAM_01 + test_recall_ResNet50_ADAM_001 + test_recall_ResNet50_ADAM_0001 +
                                 test_recall_ResNet50_SGD_01 + test_recall_ResNet50_SGD_001 + test_recall_ResNet50_SGD_0001 +
                                 test_recall_ResNet50_RMSprop_01 + test_recall_ResNet50_RMSprop_001 + test_recall_ResNet50_RMSprop_0001) / 9

average_test_cohen_kappa_ResNet50 = (test_cohen_kappa_ResNet50_ADAM_01 + test_cohen_kappa_ResNet50_ADAM_001 + test_cohen_kappa_ResNet50_ADAM_0001 +
                                   test_cohen_kappa_ResNet50_SGD_01 + test_cohen_kappa_ResNet50_SGD_001 + test_cohen_kappa_ResNet50_SGD_0001 +
                                   test_cohen_kappa_ResNet50_RMSprop_01 + test_cohen_kappa_ResNet50_RMSprop_001 + test_cohen_kappa_ResNet50_RMSprop_0001) / 9

average_test_jaccard_index_ResNet50 = (test_jaccard_index_ResNet50_ADAM_01 + test_jaccard_index_ResNet50_ADAM_001 + test_jaccard_index_ResNet50_ADAM_0001 +
                                   test_jaccard_index_ResNet50_SGD_01 + test_jaccard_index_ResNet50_SGD_001 + test_jaccard_index_ResNet50_SGD_0001 +
                                   test_jaccard_index_ResNet50_RMSprop_01 + test_jaccard_index_ResNet50_RMSprop_001 + test_jaccard_index_ResNet50_RMSprop_0001) / 9

# Average metric formulas for VGG19
average_test_accuracy_VGG19 = (test_accuracy_VGG19_ADAM_01 + test_accuracy_VGG19_ADAM_001 + test_accuracy_VGG19_ADAM_0001 +
                               test_accuracy_VGG19_SGD_01 + test_accuracy_VGG19_SGD_001 + test_accuracy_VGG19_SGD_0001 +
                               test_accuracy_VGG19_RMSprop_01 + test_accuracy_VGG19_RMSprop_001 + test_accuracy_VGG19_RMSprop_0001) / 9

average_test_precision_VGG19 = (test_precision_VGG19_ADAM_01 + test_precision_VGG19_ADAM_001 + test_precision_VGG19_ADAM_0001 +
                                test_precision_VGG19_SGD_01 + test_precision_VGG19_SGD_001 + test_precision_VGG19_SGD_0001 +
                                test_precision_VGG19_RMSprop_01 + test_precision_VGG19_RMSprop_001 + test_precision_VGG19_RMSprop_0001) / 9

average_test_recall_VGG19 = (test_recall_VGG19_ADAM_01 + test_recall_VGG19_ADAM_001 + test_recall_VGG19_ADAM_0001 +
                             test_recall_VGG19_SGD_01 + test_recall_VGG19_SGD_001 + test_recall_VGG19_SGD_0001 +
                             test_recall_VGG19_RMSprop_01 + test_recall_VGG19_RMSprop_001 + test_recall_VGG19_RMSprop_0001) / 9

average_test_cohen_kappa_VGG19 = (test_cohen_kappa_VGG19_ADAM_01 + test_cohen_kappa_VGG19_ADAM_001 + test_cohen_kappa_VGG19_ADAM_0001 +
                                   test_cohen_kappa_VGG19_SGD_01 + test_cohen_kappa_VGG19_SGD_001 + test_cohen_kappa_VGG19_SGD_0001 +
                                   test_cohen_kappa_VGG19_RMSprop_01 + test_cohen_kappa_VGG19_RMSprop_001 + test_cohen_kappa_VGG19_RMSprop_0001) / 9

average_test_jaccard_index_VGG19 = (test_jaccard_index_VGG19_ADAM_01 + test_jaccard_index_VGG19_ADAM_001 + test_jaccard_index_VGG19_ADAM_0001 +
                                   test_jaccard_index_VGG19_SGD_01 + test_jaccard_index_VGG19_SGD_001 + test_jaccard_index_VGG19_SGD_0001 +
                                   test_jaccard_index_VGG19_RMSprop_01 + test_jaccard_index_VGG19_RMSprop_001 + test_jaccard_index_VGG19_RMSprop_0001) / 9

# List of models
models = ['DenseNet201', 'InceptionV3', 'MobileNetV2', 'ResNet50', 'VGG19']

# Corresponding lists of average accuracy, precision, and recall values
average_test_accuracies = [
    average_test_accuracy_DenseNet201,
    average_test_accuracy_InceptionV3,
    average_test_accuracy_MobileNetV2,
    average_test_accuracy_ResNet50,
    average_test_accuracy_VGG19
]

average_test_precisions = [
    average_test_precision_DenseNet201,
    average_test_precision_InceptionV3,
    average_test_precision_MobileNetV2,
    average_test_precision_ResNet50,
    average_test_precision_VGG19
]

average_test_recalls = [
    average_test_recall_DenseNet201,
    average_test_recall_InceptionV3,
    average_test_recall_MobileNetV2,
    average_test_recall_ResNet50,
    average_test_recall_VGG19
]

average_test_cohen_kappas = [
    average_test_cohen_kappa_DenseNet201,
    average_test_cohen_kappa_InceptionV3,
    average_test_cohen_kappa_MobileNetV2,
    average_test_cohen_kappa_ResNet50,
    average_test_cohen_kappa_VGG19
]

average_test_jaccard_indexs = [
    average_test_jaccard_index_DenseNet201,
    average_test_jaccard_index_InceptionV3,
    average_test_jaccard_index_MobileNetV2,
    average_test_jaccard_index_ResNet50,
    average_test_jaccard_index_VGG19
]


# Plot bar graphs
x = np.arange(len(models))  # the label locations
width = 0.15  # the width of the bars

fig, ax1 = plt.subplots(figsize=(10, 6))
# Create the first y-axis for accuracy, precision, and recall
ax1.set_xlabel('Model')
ax1.set_ylabel('Scores')
ax1.set_title('Model Performance Evaluation')

# Assign different colors to each metric
colors = ['blue', 'green', 'orange', 'red', 'purple']

rects1 = ax1.bar(x - 2 * width, average_test_accuracies, width, label='Average Accuracy', color=colors[0])
rects2 = ax1.bar(x - width, average_test_precisions, width, label='Average Precision', color=colors[1])
rects3 = ax1.bar(x, average_test_recalls, width, label='Average Recall', color=colors[2])

# Create a second y-axis for Cohen's Kappa and Jaccard Index
ax2 = ax1.twinx()
ax2.set_ylabel('Scores (Cohen\'s Kappa and Jaccard Index)')

rects4 = ax2.bar(x + width, average_test_cohen_kappas, width, label='Average Cohen\'s Kappa', color=colors[3])
rects5 = ax2.bar(x + 2 * width, average_test_jaccard_indexs, width, label='Average Jaccard Index', color=colors[4])

# Combine all metrics in a single legend at the bottom
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=5)

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax1.set_xticks(x - width / 2)
ax1.set_xticklabels(models)

# Move the legend to the side
fig.tight_layout()

# Save the plot
output_directory = 'results'
os.makedirs(output_directory, exist_ok=True)
plot_filename = os.path.join(output_directory, 'average_model_performance_evaluation.png')
plt.savefig(plot_filename, bbox_inches='tight')  # Use bbox_inches to include the legends

# Show the plot
plt.show()